# Lecture 9

## Iterative Methods

### 0.Overview

    A. References 
        
        1. Golub + Valn Loan
        2. Saad 2000, Sparse System
        3. Shewchuk 1994, Intro to conjugated Gradients without the agonizing pain
        4. Gower + Richtarik Randomized Interactive Methods for Linear System 2015
    
    B. Outline
        
        1. Motivate iterative methods
        2. Splitting methods
        3. Sparse Randomized Kaczmarz Method
        4. Gradient Descent
        5. Conjugate Gradients
        6. GMRES (Gerneral...residual) (optional)

### I. Motivation

$Ax = b$, where $A$ is sparse

### II. Splitting methods

Let $x^c$ denote current iterate and $x^+$ denote next iterate. 

$r^c = Ax^c - b$ current residual; $r^+ = Ax^+ - b$ next residual.

$A = D - E - F$
    - $D$ is the diagonal entries of A
    - $E$ is the negative lower triangular matrix of A
    - $F$ is the negative upper triangular matrix of A

<font color=brown>**Example:**</font>

$\begin{bmatrix} 3 & 6 & 9\\ 2 & 4 & 8 \\ 5 & 13 & 9  \end{bmatrix} = \begin{bmatrix} 3 & 0 & 0\\ 0 & 4 & 0 \\ 0 & 0 & 9  \end{bmatrix} + \begin{bmatrix} 0 & 0 & 9\\ -2 & 0 & 0 \\ -5 & -13 & 0  \end{bmatrix} + \begin{bmatrix} 0 & -6 & -9\\ 0 & 0 & -8 \\ 0 & 0 & 0  \end{bmatrix}$


#### A. Jacobi Method

Let ${e_j}$ be the standard basis in $R^d$. 

<font color=brown>**Basic Idea:**</font>set $r_i^+$ to zero.

<font color=black>
$$0 = r_i^+ = b_i - \sum_{k = 1, k\neq i}^{d} A_{ik}x_k^c - A_{ii}x_i$$
$$ \implies A_{ii}x_i^+ = b_i - \sum_{k = 1, k\neq i}^{d} A_{ik}x_k^c $$
$$ \implies x_i^+ = \frac{1}{A_{ii}}(b_i - \sum_{k = 1, k\neq i}^{d} A_{ik}x_k^c) $$
$$ \iff x_i^+ = \frac{1}{A_{ii}}(b_i + \sum_{k = 1}^{i-1} E_{ik}x_k^c) + \sum_{k = i+1}^{d} F_{ik}x_k^c) $$
$$\iff X^+ = D^{-1}(b + (E+F)X^c)$$
</font>

#### B. Gauss-Seidel

<font color=brown>**Basic Idea:**</font>:set $r_i^+ = 0$, $x_i^+ \,\,\,from \,\,\, x_i^c$

<font color=black>
$$0 = r_i^+ = b_i -  \sum_{k = 1}^{i-1} A_{ik}x_k^+) + \sum_{k = i+1}^{d} A_{ik}x_k^c) - A_{ii}x_i$$
$$ \iff x_i^+ = \frac{1}{D_{ii}}(b_i + \sum_{k = 1}^{i-1} E_{ik}x_k^+) + \sum_{k = i+1}^{d} F_{ik}x_k^c) $$
$$\iff X^+ = D^{-1}(b + EX^+ +FX^c)$$
$$\iff X^+ = (D-E)^{-1}(b +FX^c)$$
</font>

#### C. Successive Over Relaxation

<font color=brown>**referemce:**</font>

http://www.nhcue.edu.tw/~jinnliu/teaching/nde07/Lecture6.pdf

https://math.unice.fr/~frapetti/CorsoF/cours3.pdf

<font color=brown>**Definition:**</font>

Successive Over Relaxation:
$$(D-wE){x^+} = [wF + (1-w)D]x^c+wb$$
Backward SOR: 
$$(D-wF){x^+} = [wE + (1-w)D]x^c+wb$$
Symmetric SOR
$$(D-wE){z} = [wF + (1-w)D]x^c+wb$$
$$(D-wF){x^+} = [wE + (1-w)D]z+wb$$

**HW problem:** What do we need from A to ensure that these different schemes are well-defined.

**My answer:**

- First the diagonal entries of matrix A can't be zeros. That is A has non-zero diagonal entries.
- $D - E$ need to be invertible
- $D - F$ need to be invertible
- A is square ( a system of n linear equations with unknown x)
- A needs to be invertible?

### Convergence.

$x^+ = Gx^c + f$
- Jacob: $G = D^{-1}(E+F),\,\,\, f = D^{-1}b$
- Gauss-Seidel: $G = (D-E)^{-1}F, \,\,\, f = (D-E)^{-1}b$
- Successive Over Relaxation: $G = (D-wE)^{-1}(wF + (1-w)D), \,\,\, f = w(D-wE)^{-1}b$

$x^*$ such that $Ax^* = b$
- Jacob: $f = D^{-1}b = D^{-1}Ax^* = D^{-1}(D - E - F)x^* = (I - G)x^*$
- Gauss-Seidel: $f = (D-E)^{-1}b = (D-E)^{-1}Ax^* = (I - G)x^*$
- Successive Over Relaxation: $f = w(D-wE)^{-1}b = w(D-wE)^{-1}Ax^* = (D-wE)^{-1}(wD-wE-wF)x^* = (D-wE)^{-1}(D-wE-((1-w)D +wF))x^* = (I - G)x^*$

<font color=brown>**Lemma:**</font> 
For Jacobi, GS, SOR, $\exists x^*$ s.t. $Ax^* = b$ then 
$$x^+ - x^* = G(x^c - x^*)$$

** proof: **
$$x^+ = Gx^c + f = Gx^c + (I-G)x^*$$
rearrange:
$$x^+ - x^* = G(x^c - x^*)$$


<font color=brown>**Theorem:**</font> 
Suppose $\exists x^*$ s.t. $Ax^* = b$ Let $x_0$ be arbitrary define $\{x_k, k\in N\}$ and $x_k = Gx_{k-1} + F$. If $p(G) < 1$, then $x^*$ is unique and $x_k \rightarrow x^*$

Note that: $p(G)$ is the largest module of the eigenvalues of G

**HW problem:** Does this lemma hold for SSOR?

**My answer:**
Yes, this lemma hold for SSOR.

Symmetric SOR:
$$(D-wE){z} = [wF + (1-w)D]x^c+wb$$
$$(D-wF){x^+} = [wE + (1-w)D]z+wb$$

With a little algebra, we can get:
$$x^+ = (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}[wF + (1-w)D]x^c + (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}wb + (D-wF)^{-1}wb$$
$$\iff x^+ = (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}[wF + (1-w)D]x^c + (2-w)w(D-wF)^{-1}D(D-wE)^{-1}b$$

Thus, $$G = (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}[wF + (1-w)D]$$  $$f = (2-w)w(D-wF)^{-1}D(D-wE)^{-1}b$$

or 

$$G = (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}[wF + (1-w)D]$$  $$f = (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}wb + (D-wF)^{-1}wb$$

we will use the second form to prove that if $\exists x^*$ s.t. $Ax^* = b$, then $f = (I - G)x^*$

$\begin{align} f &= \{(D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1} + (D-wF)^{-1}\}wb \\
& =  \{(D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1} + (D-wF)^{-1}\}w(D-E-F)x^* \\
& = (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}(wD-wE-wF)x^* + (D-wF)^{-1}(wD-wE-wF)x^* \\
& = (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}(-wF - (1-w)D+ D - wE)x^* + (D-wF)^{-1}(wD-wE-wF)x^* \\
& = -Gx^* + (D-wF)^{-1}[wE + (1-w)D](D-wE)^{-1}(D-wE)x^*+ (D-wF)^{-1}(wD-wE-wF)x^* \\
& = -Gx^* + (D-wF)^{-1}(wE + (1-w)D + wD -wE-wF)x^* \\ 
& = (I - G)x^*
\end{align}$

Thus, the lemma holds for SSOR

<font color=brown>**Definition: Jordan Canonical Form**</font>
A suqre matrix $A \in R^{n \times n}$. $\exists X$ invertible and a block diag matrix $J$ whose blocks are $\lambda I + E$, where $\lambda$ is an eigenvalue of A and $E = \{ \begin{array}{rl} 1, & j = i+1 \\ 0, & \text{ otherwise} \end{array} $ such that $A = XJX^{-1}$

**HW problem:** Compute $J^k$ where $J = \lambda I + E$. Show that if $G = XJX^{-1}$ and $P(G) < 1$ then $J^k \rightarrow 0$ as $k \rightarrow \infty$

**My answer:**

From previous lemma and theorem we know that:
$$x_k \rightarrow x^*$$ 
$$x^{k+1} - x^* = G(x^k - x^*)$$
which means that:
$G^k \rightarrow 0$ as $k \rightarrow \infty$ 

However,
$$G^k = (XJX^{-1})^k = XJ^kX^{-1}$$
Thus,
$G^k \rightarrow 0$ as $k \rightarrow \infty$ $\iff$ $J^k \rightarrow 0$ as $k \rightarrow \infty$